In [1]:
from src.dataset2 import *
from src.model5 import *
from src.train import *
from src.graph import *

import torch
import src.templates
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "template_TNR"

C:\Users\alwas\Desktop\egnn_tda\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = QM9Dataset(root="./dataset/qm9+tf",
                     target_y="gap",
                     max_atoms=100,
                     force_reload=True
                     )

Processing...
  0%|          | 0/133885 [00:00<?, ?it/s]C:\Users\alwas\Desktop\egnn_tda\.venv\Lib\site-packages\ripser\ripser.py:251: UserWarning: The input matrix is square, but the distance_matrix flag is off.  Did you mean to indicate that this was a distance matrix?
  warnings.warn(
C:\Users\alwas\Desktop\egnn_tda\.venv\Lib\site-packages\ripser\ripser.py:251: UserWarning: The input matrix is square, but the distance_matrix flag is off.  Did you mean to indicate that this was a distance matrix?
  warnings.warn(
100%|██████████| 133885/133885 [04:17<00:00, 519.54it/s]
Done!


In [4]:
visualize_pi(dataset[458].PI[0])

In [ ]:
Rips_complex = gd.RipsComplex(points = dataset[115497].pos, max_edge_length=5)
Rips_simplex_tree = Rips_complex.create_simplex_tree(max_dimension=3)
diag_Rips = Rips_simplex_tree.persistence()
gd.plot_persistence_diagram(diag_Rips, legend=True)

In [ ]:
import numpy as np
from ripser import ripser

# --- optional: gaussian blur (preferred) ---
try:
    from scipy.ndimage import gaussian_filter
    _HAS_SCIPY = True
except Exception:
    _HAS_SCIPY = False

def gaussian_blur_2d(img, sigma_px=1.0):
    """Gaussian blur in pixel units. Uses SciPy if available, else a small NumPy fallback."""
    if sigma_px <= 0:
        return img

    if _HAS_SCIPY:
        return gaussian_filter(img, sigma=sigma_px, mode="constant")

    # --- NumPy fallback: separable convolution with truncated Gaussian kernel ---
    radius = int(np.ceil(3.0 * sigma_px))
    x = np.arange(-radius, radius + 1)
    k = np.exp(-(x**2) / (2.0 * sigma_px**2))
    k = (k / k.sum()).astype(np.float32)

    # convolve rows then cols (separable)
    tmp = np.apply_along_axis(lambda m: np.convolve(m, k, mode="same"), axis=1, arr=img)
    out = np.apply_along_axis(lambda m: np.convolve(m, k, mode="same"), axis=0, arr=tmp)
    return out.astype(np.float32)

# --- your settings ---
index = 0
maxdim = 2
thresh = 10

bins = 64
x_min, x_max = 0.0, 5.0
y_min, y_max = 0.0, 5.0

# gaussian blur strength in "pixels"
sigma_px = 1.5  # try 0.7–1.5; 0 means no blur

dgms = ripser(dataset[index].pos, maxdim=maxdim, thresh=thresh)["dgms"]
dgms = [dgms[i][np.isfinite(dgms[i][:, 1])] for i in range(maxdim + 1)]

PI = np.zeros((maxdim + 1, bins, bins), dtype=np.float32)

for i in range(maxdim + 1):
    if dgms[i].shape[0] == 0:
        continue

    birth = dgms[i][:, 0]
    death = dgms[i][:, 1]

    # rotate by -pi/4 (and scale): (b,d) -> (x,y)
    #x = (birth + death) / np.sqrt(2.0)
    #y = (death - birth) / np.sqrt(2.0)  # ~ persistence

    x = birth + 1
    y = death - birth + 1

    # weights (optional). If you want weighting: weights=w; if not: weights=None
    w = y  # or y**1.0

    H, xedges, yedges = np.histogram2d(
        x, y,
        bins=bins,
        range=[[x_min, x_max], [y_min, y_max]],
        weights=None  # <- set to w if you want persistence-weighted histogram
    )

    # transpose to make axis0=y, axis1=x (for imshow with origin="lower")
    img = H.T.astype(np.float32)

    # --- Gaussian blur (key step) ---
    img = gaussian_blur_2d(img, sigma_px=sigma_px)

    # Optional: L1-normalize
    #s = img.sum()
    s = 1
    if s > 0:
        img /= s

    PI[i] = img

print(PI.shape, PI.min(), PI.max())


In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# PI: (maxdim+1, bins, bins)
# x_min, x_max, y_min, y_max, maxdim должны быть определены выше

ncols = maxdim + 1
bins = PI.shape[1]

dx = (x_max - x_min) / bins
dy = (y_max - y_min) / bins

fig = make_subplots(
    rows=1, cols=ncols,
    subplot_titles=[f"PI (H{i})" for i in range(ncols)],
    horizontal_spacing=0.06
)

for i in range(ncols):
    fig.add_trace(
        go.Heatmap(
            z=PI[i],
            x0=x_min, dx=dx,
            y0=y_min, dy=dy,
            colorscale="Viridis",
            colorbar=dict(title="value", len=0.85)  # each subplot gets its own bar
        ),
        row=1, col=i+1
    )

fig.update_layout(
    width=350*ncols,
    height=380,
    margin=dict(l=10, r=10, t=40, b=10)
)

fig.show()


## TDA

In [3]:
model5 = E3GG(node_attr_dim = dataset.node_attr.shape[1],
              edge_dim   = dataset.edge_attr.shape[1],
              hidden_dim = 64,
              num_layers=7
              )

In [4]:
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
import torch

# split
n = len(dataset)
n_train = int(0.90 * n)
n_test = n - n_train
train_ds, test_ds = random_split(dataset, [n_train, n_test], generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=128, shuffle=False)

In [5]:
n_epochs = 500
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

optimizer = torch.optim.Adam(model5.parameters(), lr=1e-3, weight_decay=1e-16)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs)

train_losses, val_losses = [], []

In [6]:
model5.to(device)

_train_losses, _val_losses = train( model = model5,
                                  train_loader = train_loader,
                                  val_loader = test_loader,
                                  optimizer = optimizer,
                                  scheduler = lr_scheduler,
                                  epochs = n_epochs,
                                  device = device,
                                  )

train_losses += _train_losses
val_losses += _val_losses

100%|██████████| 500/500 [3:45:23<00:00, 27.05s/it]  


In [7]:
fig = go.Figure(layout={
        'plot_bgcolor': 'white',
        'paper_bgcolor' : 'white',})

fig.update_layout(width = 600,
                  height = 600,
                  legend = dict(x = 0.95, y = 0.9)
                  )

fig.update_xaxes(title = "epoch")
fig.update_yaxes(title = "MSE loss")

fig.add_trace(go.Scatter(y=val_losses, mode='lines', name = 'Validation loss'))
fig.add_trace(go.Scatter(y=train_losses, mode='lines', name = 'Training loss'))

In [8]:
import src.templates
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "template_TNR"

model5.to("cpu")
model5.eval()

fig = go.Figure(layout={
        'plot_bgcolor': 'white',
        'paper_bgcolor' : 'white',})

fig.update_layout(width = 600,
                  height = 600
                  )
fig.update_xaxes(title = "model",
                 range=[0, 15]
)
fig.update_yaxes(title = "dataset",
                 range=[0, 15])

fig.add_trace(go.Scatter(x = [-15, 15], y = [-15, 15], mode = 'lines', line=dict(color = "lightgrey"), showlegend=False))

for batch in tqdm(train_loader):
    fig.add_trace(go.Scatter(x = model5(batch).detach(),
                             y = batch.y,
                             mode = 'markers',
                             marker = dict(color = "blue"),
                             showlegend=False,
                             )
                  )

for batch in tqdm(test_loader):
    fig.add_trace(go.Scatter(x = model5(batch).detach(),
                             y = batch.y,
                             mode = 'markers',
                             marker = dict(color = "green"),
                             showlegend=False,
                             )
                  )

fig.show()

100%|██████████| 103/103 [00:05<00:00, 19.57it/s]


In [ ]:
torch.save(model5.state_dict(), "model5.pt")

In [10]:
MAE_train = []
MAE_valid = []

for batch in train_loader:
    MAE_train.append( (np.abs(model5(batch).detach() -  batch.y)).numpy() )

for batch in test_loader:
    MAE_valid.append( (np.abs(model5(batch).detach() -  batch.y)).numpy() )

C:\Users\alwas\AppData\Local\Temp\ipykernel_19000\1794734912.py:5: DeprecationWarning:

__array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)

C:\Users\alwas\AppData\Local\Temp\ipykernel_19000\1794734912.py:8: DeprecationWarning:

__array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)



In [11]:
print(f"MAE (train): {1000 * np.mean(np.concatenate(MAE_train)):.2f} meV")
print(f"MAE (valid): {1000 * np.mean(np.concatenate(MAE_valid)):.2f} meV")

MAE (train): 49.40 meV
MAE (valid): 113.05 meV


In [19]:
np.sum([p.numel() for p in model5.parameters()])

np.int64(245503)